In [1]:
import freenect

In [29]:
import freenect
import cv2
import numpy as np

# --- SETTINGS ---

# 1. Smoothing
# Kernel size for the median blur (must be an odd number)
MEDIAN_BLUR_KERNEL = 25 

# 2. Topography
# The "height" of each elevation step (smaller = more lines/colors)
CONTOUR_STEP = 2
# Colormap for the "land"
LAND_COLORMAP = cv2.COLORMAP_JET # (blue-green-yellow-red)

# 3. Water
# Any pixel with a depth value *below* this will be water.
# Tune this value (0-255) to match your sandbox's base level.
SEA_LEVEL = 100
# Color of the water (in BGR format)
WATER_COLOR = [255, 0, 0] # Bright Blue

# --- KINECT FUNCTIONS ---

def get_depth():
    """
    Gets 8-bit normalized (0-255) depth data from the Kinect.
    Invalid pixels (occlusions) are set to 255.
    """
    array, _ = freenect.sync_get_depth() 
    np.clip(array, 0, 1023, out=array)
    array >>= 2
    array = array.astype(np.uint8)
    return array

def get_video():
    """
    Gets 8-bit BGR (OpenCV-ready) video data from the Kinect.
    """
    array, _ = freenect.sync_get_video()
    array = cv2.cvtColor(array, cv2.COLOR_RGB2BGR) 
    return array

# --- MAIN PROGRAM ---

if __name__ == "__main__":
    while True:
        # --- 1. ACQUISITION ---
        # Get raw depth for masking and RGB for display
        depth_raw = get_depth()
        rgb = get_video()
        
        # --- 2. SMOOTHING ---
        # Apply a median blur to remove "salt-and-pepper" noise
        depth_smooth = cv2.medianBlur(depth_raw, MEDIAN_BLUR_KERNEL)
        
        # --- 3. ANALYSIS & VISUALIZATION ---
        
        # A. Create discrete elevation "steps"
        quantized_depth = (depth_smooth // CONTOUR_STEP) * CONTOUR_STEP
        
        # B. Apply colormap to the quantized "land"
        depth_colormap = cv2.applyColorMap(quantized_depth, LAND_COLORMAP)
        
        # C. Find contour lines (the edges of the "steps")
        edges = cv2.Canny(quantized_depth, 100, 200)

        # D. Draw contours (black lines) onto the colormap
        depth_colormap[edges == 255] = [0, 0, 0] 
        
        # E. Add "water" by coloring all pixels below sea level
        # We use the smoothed depth map for this check
        depth_colormap[depth_smooth < SEA_LEVEL] = WATER_COLOR
        
        # --- 4. MASKING ---
        
        # Find all "invalid" pixels (occlusion shadows)
        # (These are pixels where raw depth was 2047, which became 255)
        invalid_mask = (depth_raw == 255)
        
        # Apply the mask to the final image
        depth_colormap[invalid_mask] = [0, 0, 0]
        
        # --- 5. DISPLAY ---
        
        # Show the debug RGB stream
        cv2.imshow('Kinect RGB (Debug)', rgb)
        
        # Show the final, processed AR Sandbox output
        cv2.imshow('AR Sandbox Output', depth_colormap) 
        
        # Wait for 1ms and check if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources and destroy windows
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to target thread (0x5f814ffcd370)

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to target thread (0x5f814ffcd370)

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to target thread (0x5f814ffcd370)

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to target thread (0x5f814ffcd370)

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to target thread (0x5f814ffcd370)

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to target thread (0x5f814ffcd370)

QObject::moveToThread: Current thread (0x5f814ffcd370) is not the object's thread (0x5f8150a7f7b0).
Cannot move to tar

In [38]:
import numpy as np
import warnings

try:
    import freenect 
    FREENECT_AVAILABLE = True
except ImportError:
    warnings.warn("Freenect module not found. The code below is for illustration only.")
    FREENECT_AVAILABLE = False

class KinectV1Simulator:
    """
    Simulates the KinectV1 class structure and frame acquisition using freenect.
    NOTE: Requires the 'freenect' package and a connected Kinect V1 sensor.
    """
    def __init__(self, sensor_id=0):
        self.name = 'kinect_v1'
        self.id = sensor_id
        # Hardcoded resolutions for Kinect V1
        self.depth_width = 320
        self.depth_height = 240
        
        if FREENECT_AVAILABLE:
            print(f"KinectV1 initialized. Ready to use sensor ID: {self.id}")
            warnings.warn("Be sure no other kernel is running before using KinectV1 to avoid a kernel crash, as two processes cannot access the Kinect simultaneously. [cite: 1740-1741]")

    def get_depth_frame(self):
        """
        Grabs the depth frame in millimeters (mm) from the Kinect V1.
        (Mimics sandbox/sensor/kinectV1.py:get_frame)
        """
        if not FREENECT_AVAILABLE:
            # Placeholder for simulation if freenect is missing
            return np.zeros((self.depth_height, self.depth_width), dtype=np.uint16)

        # 1. Synchronously get the depth data in millimeters
        # The [0] is to extract the depth array from the (data, timestamp) tuple
        depth_array = freenect.sync_get_depth(index=self.id, format=freenect.DEPTH_MM)[0] 
        
        # 2. Flip the array to correct the orientation (if necessary for your setup)
        # This is used in the original project to correct the frame 
        depth_array = np.fliplr(depth_array)
        
        return depth_array

    def get_color_frame(self):
        """
        Grabs the color (video) frame from the Kinect V1.
        (Mimics sandbox/sensor/kinectV1.py:get_color)
        """
        if not FREENECT_AVAILABLE:
            # Placeholder for simulation if freenect is missing
            return np.zeros((self.color_height, self.color_width, 3), dtype=np.uint8)

        # 1. Synchronously get the video data
        color_array = freenect.sync_get_video(index=self.id)[0]
        
        # 2. Flip the array to correct the orientation
        color_array = np.fliplr(color_array)
        
        return color_array

# --- Example Usage on your local machine ---
if __name__ == "__main__":
    if FREENECT_AVAILABLE:
        try:
            kinect = KinectV1Simulator(sensor_id=0)
            
            depth_data = kinect.get_depth_frame()
            print(f"Depth frame acquired: Shape {depth_data.shape}, Min: {depth_data.min()}, Max: {depth_data.max()}")

            color_data = kinect.get_color_frame()
            print(f"Color frame acquired: Shape {color_data.shape}")

        except Exception as e:
            print(f"An error occurred during Kinect operation: {e}")
            print("Ensure the Kinect is plugged in and drivers are loaded.")
    else:
        print("Cannot run freenect code. Please ensure the library is installed.")

KinectV1 initialized. Ready to use sensor ID: 0


/home/geologydept/anaconda3/envs/sandbox-env/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Be sure no other kernel is running before using KinectV1 to avoid a kernel crash, as two processes cannot access the Kinect simultaneously. [cite: 1740-1741]


Depth frame acquired: Shape (480, 640), Min: 0, Max: 958
Color frame acquired: Shape (480, 640, 3)
